**Copyright: © NexStream Technical Education, LLC**.
All rights reserved

#Bayes Theorem - Part 1

Create a Colab script which calculates P(A|B) using Bayes theorem for the example provided in the lecture; that is, if you test positive how probable is it that you have cancer?
- Create a Python script to implement the bayes_theorem function
- Create a runner to call your function and output the result, P(A|B) for the following
Cases:
-  Case 1: Cancer occurs in 1% of the people your age and the test is 99% reliable
-  Case 2: Cancer occurs in 3% of the people your age and the test is 90% reliable
-  Repeat the tests 3 times each (using your previous P(A|B) result in the
subsequent run) and record P(A|B) for each repeated test. How did your results
improve with each additional test run?

Summary of the lecture example:
- P(A):  probability you have cancer prior to getting tested (called the 'belief')
- P(B):  probability you test positive  (called the 'evidence')
- P(A|B):  probability you have cancer given the test is positive

- Bayes Theorem:
$$P(A|B)=\frac{P(B|A)\cdot P(A)}{P(B)}$$

<br>

**def bayes_theorem(p_a, p_b_given_a):**
- Inputs:
  - *p_a* = probability of event 'a', e.g., you have cancer prior to getting tested
  - *p_b_given_a* = probability of event 'b' given event 'a' has occurred, e.g. the probability that you test positive given you have cancer.
- return *p_a_given_b* = probability of event 'a' given event 'b', e.g. the probability that you have cancer given you test positive, *p_b* (for test purposes).

<br>

Follow the steps as outlined in the following code cell to implement the functions.  Make sure your code passes the embedded doctests.


In [8]:
#In-class assignment in Bayes lecture
#If you test positive how probable is it that you have cancer?
import numpy as np


#Calculate P(A|B) given P(A), P(B|A)
#    Hint:  calculate false_positives, true_positives, and P(B)
#           then can use Bayes theorem to calculate P(A|B)

def bayes_theorem(p_a, p_b_given_a):

    p_not_a = 1 - p_a
    p_b_given_not_a = 1 - p_b_given_a
    true_positives = p_b_given_a * p_a
    false_positives = p_b_given_not_a * p_not_a
    p_b = true_positives + false_positives

    # Calculate P(A|B) using Bayes theorem
    p_a_given_b = p_b_given_a * p_a / p_b

    return p_b, p_a_given_b
#-------------------------------------------------------------------------------------------------
#Test with the following doctest test vectors.
#DO NOT EDIT THE TEST CODE!!!!

#Scenario and definitions
#   P(A):  probability that event (e.g. cancer) occurs in % of population (belief)
#   P(B):  probability test is positive (evidence)
#   P(B|A):  probability test is positive if event occurs, i.e. reliability of test
#            when multiply by P(A) then get the rate of true positives
#   P(B|not A): probability test is positive if event does not occur
#            when multiply by P(not A) then get the rate of false positives

#Case 1.  Cancer occurs in 1% of the people your age and the test is 99% reliable
#Case 2.  Cancer occurs in 3% of the people your age and the test is 90% reliable

def bayes_runner(p_a, p_b_given_a, iterations):
  pagb_list = []
  pb_list = []
  for i in range(iterations):
    p_b, p_a_given_b = bayes_theorem(p_a, p_b_given_a)
    p_a = p_a_given_b
    pagb_list.append(p_a_given_b)
    pb_list.append(p_b)
  return pb_list, pagb_list

#Case 1.  Cancer occurs in 1% of the people your age and the test is 99% reliable
pb_1, pagb_1 = bayes_runner(0.01, 0.99, 3)

#Case 2.  Cancer occurs in 3% of the people your age and the test is 90% reliable
pb_2, pagb_2 = bayes_runner(0.03, 0.90, 3)

import doctest
"""
   >>> print(np.round(pagb_1, 4))
   [0.5    0.99   0.9999]
   >>> print(np.round(pb_1, 4))
   [0.0198 0.5    0.9802]
   >>> print(np.round(pagb_2, 4))
   [0.2177 0.7147 0.9575]
   >>> print(np.round(pb_2, 4))
   [0.124  0.2742 0.6718]
"""
doctest.testmod()


TestResults(failed=0, attempted=4)

# Naive Bayes Monomial Classifier - Part 2

Create a Colab script which uses a Naive Bayes classifier to detect email spam messages. Note, you may NOT use any machine learning libraries (e.g. sklearn) for this section.
- Create a training set of Wanted and Unwanted messages
- Create your 'model'
 - Parse the words in the emails and create conditional probabilities for the Wanted
and Unwanted messages
- Test your model with other messages not used in the training set
- Record and reflect on your results, i.e. evaluate the performance, and compare against
typical performance of the algorithm (do some searches to find the average performance
of a Naive Bayes based spam filter).
- A training dataset has been provided as materials for this assignment. You may find another, or scrape your own emails if you prefer - just describe what dataset you used or created in your reflections.


<br>

- Follow the steps as outlined in the following code cells to implement the functions.
- Record your comments and reflections for this part in a text cell at the end of this section.



In [9]:
#Mount your google drive and copy the dataset to the current working directory (!cp),
#or change the working directory to the folder (%cd).
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/colabfiles


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/colabfiles


In [53]:
#Read the dataset as a Pandas dataframe.
#and examine the head of the file.
#An example is shown for the provided dataset.
#This code assumes the dataset csv file is in the working directory.

import pandas as pd

df = pd.read_csv('spam_training_dataset_2.csv')
df['text'].str.lower()
df['text'].str.replace(r'\d+', '')
df['text'].str.replace(r'\s+', ' ')

df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [54]:
#Setup dataframes with 80% train and 20% test
#An example is shown for the provided dataset.
cutoff = int(len(df)*0.8)
train = df[:cutoff]
test = df[cutoff:]

#Reset the indices of the test rows
test.reset_index(drop=True, inplace=True)


In [55]:
#Train the model

#Create empty dictionaries of ham (wanted), and spam (unwanted) messages.
#This will create the histogram bins needed for ham and spam messages.
#The dicts will set keys = email word, value = word frequency (count)
ham = {}
spam = {}

#Loop over length of the training set
for i in range(len(train)):
  #Read an example (X) == row from the training set
  #Hint:  https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html
  X = train.loc[i]

  #If the exammple is ham (can get this from the 'label' or 'label_num' column)
  #then set a reference to the ham dictionary, else set reference to spam dict.
  if X['label'] == 'ham':
    toAppend = ham
  else:
    toAppend = spam


  #Loop over the words in the example.
  #Hint: https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html
  #Hint: In this case str is the X.text field in the df
  for word in X['text'].split():

    #Check for 'stopwords' by ignoring any words with length < X (try different lengths)
    if len(word) > 4:
      #Check if word is in the dictionary and increment count if so
      #otherwise add the word to the dictionary and set the count to 1
      if word in toAppend:
        toAppend[word] += 1
      else:

        toAppend[word] = 1

#Set the lengths for each of the ham and spam entries
#so can calculate the probability of each word in the ham and spam histograms.
lenHam = sum(ham.values())
lenSpam = sum(spam.values())

#Loop over each word and frequency in the dicts and calculate their weightings.
#Hint:  https://docs.python.org/3/tutorial/datastructures.html#dictionaries (Looping Techniques)
#Ham histogram probabilities
for word, freq in ham.items():
  ham[word] = freq/lenHam
#Spam histogram probabilities
for word, freq in spam.items():
  spam[word] = freq/lenSpam


#Finally sort the dictionaries by values in decending order (highest weighting first)
#and keep the items with the top 25 weightings.
#Note, you should experiment with keeping different amounts of items.
ham = dict(sorted(ham.items(), key=lambda item: item[1], reverse=True)[:10000])
print(ham)
spam = dict(sorted(spam.items(), key=lambda item: item[1], reverse=True)[:10000])
print(spam)

{'&lt;#&gt;': 0.012812690665039659, 'going': 0.006434078429197404, 'about': 0.006378612235842254, 'still': 0.005657551722225303, "don't": 0.004991957401963503, 'there': 0.004936491208608353, 'think': 0.004936491208608353, 'later': 0.0038271673415053525, '&amp;': 0.0037717011481502023, 'should': 0.003327971601309002, 'would': 0.003106106827888402, 'really': 0.002995174441178102, 'great': 0.0028842420544678018, 'after': 0.0028842420544678018, 'night': 0.002773309667757502, 'right': 0.0026623772810472016, 'doing': 0.0026623772810472016, 'lor...': 0.0026623772810472016, 'phone': 0.0026623772810472016, 'gonna': 0.0025514448943369018, 'where': 0.0025514448943369018, 'Sorry,': 0.0024959787009817516, 'Happy': 0.0024959787009817516, 'around': 0.0023295801209163016, "can't": 0.0023295801209163016, 'could': 0.0023295801209163016, 'always': 0.002163181540850851, 'tomorrow': 0.0021077153474957015, 'today': 0.0021077153474957015, 'already': 0.0020522491541405514, 'getting': 0.0020522491541405514, 'S

In [58]:
#Make predictions
#def predict(text):
#Input: test message text
#Return: 'ham' or 'spam' label

# Loop over words in test message
#   if not stopword (<4 char) then accumulate wanted or unwanted probabilities
# Note, for this implementation, if the word is not in the ham or spam dictionary,
# just ignore it, i.e. don't multiply in a 0 probability to avoid the zero frequency problem.
# If the wanted score > unwanted score then return 'ham', else return 'spam'

#YOUR CODE HERE

def predict(text):
  w_score = 1.0
  u_score = 1.0
  for word in text.split():
    if len(word) > 4:
      if word in ham:
        w_score = w_score * ham[word]
      if word in spam:
        u_score = u_score * spam[word]

  #Compare w_score against u_score and return 'ham' or 'spam' label
  if w_score > u_score:
    return 'ham'
  else:
    return 'spam'


In [59]:
#Test the model
#Loop over the test dataset
#  extract an example (row) and pass the text field to the predict function
#  if the returned label matches the example label then increment the correct
#Calculate a percent correct

#YOUR CODE HERE
# Use test.loc[i] to extract each row from the test dataset.
# Pass the 'text' field of the extracted row to the predict function.
# Compare the prediction with the actual label in the test data.
# Increment 'correct' by 1 when the prediction matches the actual label.
# Use len(test) to get the total number of test examples.
correct = 0
for i in range(len(test)):
  email = test.loc[i, 'text']
  if predict(email) == test.loc[i,'label']:
    correct += 1


print(correct/len(test))

0.0862533692722372


**Enter your model performance and reflections here.**
.
.
.




# Naive Bayes Gaussian Classifier (from scratch) - Part 3

Create a Colab script which uses a Naive Bayes classifier to predict one of the 3 species of flowers based on input attributes from the Iris dataset.
Note, you may NOT use any machine learning libraries (e.g. sklearn) for this section.
- Create an 80/20 split of the dataset (provided)
- Generate the Gaussian distributions for each of your attributes in each of the classes (species) for your training data
- Use the test data to predict the species
- Calculate and record the accuracy of your classifier
- The Iris dataset has been provided as materials for this assignment, however for testing purposes, please run the provided cell which accesses the dataset directly and performs a train/test split on the data (this is the only use of a sklearn function in this section).

Summary of lecture notes:
- Generate an initial guess (prior probabilities) for each of the classes based on the number of examples for each class, e.g.
 - P(Setosa) = 50/150
 - P(Versicolor) = 50/150
 - P(Virginica) = 50/150
Note, depending on your train/test split, the initial guess probabilities may be different
- Calculate the score for each of the classes, e.g.

*SetosaScore =  P(Setosa) * g(SepalLength | Setosa) * g(SepalWidth | Setosa) * g(PetalLength | Setosa) * g(PetalWidth | Setosa)*
etc.

where:
$$g(x)= \frac{1}{\sigma \sqrt(2\pi)}exp(-\frac{1}{2}\frac{(x-\mu)^2}{\sigma^2}) $$



In [60]:
#Load the dataset into a Pandas dataframe and split the data.
#The cell has been provided to set up a random seed for testing purposes.
#Please do NOT alter the code.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(csv_url, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])
#print(df.head())


feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
for col in feature_cols:
  df[col] = pd.to_numeric(df[col])

X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df['species'], test_size=0.2, random_state=1)
#print(X_train)
#print(y_train)

In [62]:
#Calculate the mean and standard deviation of the training set
#Hint: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
#Hint: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html


train_mean = X_train.groupby(y_train).mean()
train_std = X_train.groupby(y_train).std()


#-------------------------------------------------------------------------------------------------
#Test with the following doctest test vectors.
#DO NOT EDIT THE TEST CODE!!!!


import doctest
"""
   >>> print(np.round(train_mean['petal_length']['Iris-setosa'], 4))
   1.4692
   >>> print(np.round(train_std['petal_length']['Iris-setosa'], 4))
   0.1608
   >>> print(np.round(train_mean['sepal_width']['Iris-virginica'], 4))
   2.9523
   >>> print(np.round(train_std['sepal_width']['Iris-virginica'], 4))
   0.3069
"""
doctest.testmod()


TestResults(failed=0, attempted=4)

In [65]:
#Prediction
#  def predict(example):
#    init best_score and best_species
#    loop over each of the species
#       set initial species_score (prior probability)
#       loop over the feature_cols (defined in the provided cell)
#           extract the mean and std for the species feature
#           calculate the species_score (accumulate probability products)
#       if species_score > best_score
#           update best_score and best_species
#     return best_species

import math

def predict(example):
  best_score = float('-inf')
  best_species = None
  for species in df['species'].unique():
    species_score = 1/3
    for col in feature_cols:
      s = train_std[col][species]
      m = train_mean[col][species]

      exponent = -0.5 * ((example[col]-m) / s)**2
      coeff = 1 / (s * math.sqrt(2*math.pi))
      probability = coeff * math.exp(exponent)
      species_score = species_score * probability
    if species_score > best_score:
      best_species = species
      best_score = species_score

  return best_species

In [66]:
#Test the classifier

#-------------------------------------------------------------------------------------------------
#Test with the following doctest test vectors.
#DO NOT EDIT THE TEST CODE!!!!

t = 0
for i in range(len(X_test)):
  test_example = X_test.iloc[i]
  #print(test_example)
  pred = predict(test_example)
  actual = y_test.iloc[i]
  if pred == actual:
    t+=1


import doctest
"""
   >>> print(np.round(t/len(X_test), 4))
   0.9667
"""

doctest.testmod()


TestResults(failed=0, attempted=1)

# Naive Bayes Gaussian Classifier (sklearn) - Part 4

Repeat Part 3 using Scikit-learn functions:

Hint: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

#Note, use the X_train, X_test, y_train, y_test are from the provided
#cell in Part 3

#Instantiate the Naive Bayes classifier
#Fit the model
#Run the predictions
#Calculate the score


clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)



#-------------------------------------------------------------------------------------------------
#Test with the following doctest test vectors.
#DO NOT EDIT THE TEST CODE!!!!

import doctest
"""
   >>> print(np.round(clf.score(X_test, y_pred), 4))
   1.0
"""

doctest.testmod()


TestResults(failed=0, attempted=1)